In [2]:
from feature_extraction_pipeline.open_data_marine_weather.core import MarineWeatherExtractor
from feature_extraction_pipeline.open_data_raw_material_price.core import RawMaterialPriceExtractor
from feature_extraction_pipeline.open_data_terrestrial_weather.core import TerrestrialWeatherExtractor
import pandas as pd
import numpy as np



def load_data(date: str):
    price, p_key = RawMaterialPriceExtractor(date=date).process()
    t_weather, t_key = TerrestrialWeatherExtractor(date=date).process()
    m_weather, m_key = MarineWeatherExtractor(date=date).process()

    # combine data
    weather = pd.merge(
        t_weather.groupby(["일시"]).mean(),
        m_weather.groupby(["일시"]).mean(),
        how='inner', left_on=t_key, right_on=m_key
    ).reset_index()

    return pd.merge(
        price, weather, 
        how="left", left_on=p_key, right_on=t_key
    ).drop("일시", axis=1).astype(dtype={"조사일자": "datetime64"})

data201908 = load_data("201908")
data201907 = load_data("201907")

dataset = data201908.append(data201907).sort_values(by="조사일자", ascending=True)

2020-03-30 05:53:21:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/process/csv/201908.csv in s3 'production-bobsim'
2020-03-30 05:53:21:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/process/csv/201908.csv in s3 'production-bobsim'
2020-03-30 05:53:21:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/process/csv/201908.csv in s3 'production-bobsim'
2020-03-30 05:53:30:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/process/csv/201907.csv in s3 'production-bobsim'
2020-03-30 05:53:30:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/process/csv/201907.csv in s3 'production-bobsim'
2020-03-30 05:53:31:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/process/csv/201907.csv in s3 'production-bobsim'


## subtract one day and one day after

In [ ]:
dataset.